# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [ ]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [ ]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [ ]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    if hyperparams['use_pca']:
        # ...
        X_tr = X_train.copy()
        X_v = X_val.copy()
        if hyperparams['pca_standardize']:
            from sklearn.preprocessing import StandardScaler
            scaler_pca = StandardScaler().fit(X_tr)
            X_tr = scaler_pca.transform(X_tr)
            X_v = scaler_pca.transform(X_v)
        from sklearn.decomposition import PCA
        pca = PCA(n_components=hyperparams['pca_components']).fit(X_tr)
        X_tr = pca.transform(X_tr)
        X_v = pca.transform(X_v)
    else:
        X_tr = X_train.copy()
        X_v = X_val.copy()
    if hyperparams['data_standardize']:
        # ...
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler().fit(X_tr)
        X_tr = scaler.transform(X_tr)
        X_v = scaler.transform(X_v)
    # Aggiunge il termine costante ai dati
    X_tr = np.hstack([np.ones((X_tr.shape[0], 1)), X_tr])
    X_v = np.hstack([np.ones((X_v.shape[0], 1)), X_v])
    l2 = hyperparams['reg_lambda'] if hyperparams['use_regularization'] else 0.0
    I = np.eye(X_tr.shape[1])
    I[0, 0] = 0
    w = np.linalg.inv(X_tr.T @ X_tr + l2 * I) @ X_tr.T @ y_train
    from sklearn.metrics import mean_absolute_error
    y_pred_val = X_v @ w
    mae = mean_absolute_error(y_val, y_pred_val)
    return mae

# Dividi il training set in training set effettivo e validation set
# ...
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
X, y = shuffle(X, y, random_state=42)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Trova la configurazione di iperparametri migliore
# ...
best_cfg = None
best_mae = np.inf
for cfg in configs:
    mae_val = pipeline(X_train, y_train, X_val, y_val, cfg)
    if mae_val < best_mae:
        best_mae = mae_val
        best_cfg = cfg

# Riallena il modello sul training set completo
# ...
X_full = np.concatenate([X_train, X_val], axis=0)
y_full = np.concatenate([y_train, y_val], axis=0)
if best_cfg['use_pca']:
    if best_cfg['pca_standardize']:
        from sklearn.preprocessing import StandardScaler
        scaler_pca_full = StandardScaler().fit(X_full)
        X_full_proc = scaler_pca_full.transform(X_full)
    else:
        X_full_proc = X_full.copy()
    from sklearn.decomposition import PCA
    pca_full = PCA(n_components=best_cfg['pca_components']).fit(X_full_proc)
    X_full_proc = pca_full.transform(X_full_proc)
else:
    X_full_proc = X_full.copy()
if best_cfg['data_standardize']:
    from sklearn.preprocessing import StandardScaler
    scaler_full = StandardScaler().fit(X_full_proc)
    X_full_proc = scaler_full.transform(X_full_proc)
X_full_proc = np.hstack([np.ones((X_full_proc.shape[0], 1)), X_full_proc])
l2_full = best_cfg['reg_lambda'] if best_cfg['use_regularization'] else 0.0
I_full = np.eye(X_full_proc.shape[1])
I_full[0, 0] = 0
w_full = np.linalg.inv(X_full_proc.T @ X_full_proc + l2_full * I_full) @ X_full_proc.T @ y_full

# Calcola il MAE sul test set
# ...
if best_cfg['use_pca']:
    X_test_proc = X_test.copy()
    if best_cfg['pca_standardize']:
        from sklearn.preprocessing import StandardScaler
        scaler_pca_apply = StandardScaler().fit(X_full)
        X_test_proc = scaler_pca_apply.transform(X_test_proc)
    from sklearn.decomposition import PCA
    pca_apply = PCA(n_components=best_cfg['pca_components']).fit(X_full if not best_cfg['pca_standardize'] else scaler_pca_apply.transform(X_full))
    X_test_proc = pca_apply.transform(X_test_proc)
else:
    X_test_proc = X_test.copy()
if best_cfg['data_standardize']:
    from sklearn.preprocessing import StandardScaler
    scaler_apply = StandardScaler().fit(X_full_proc[:, 1:])
    X_test_proc = scaler_apply.transform(X_test_proc)
X_test_proc = np.hstack([np.ones((X_test_proc.shape[0], 1)), X_test_proc])
y_pred_test = X_test_proc @ w_full
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, y_pred_test)

# Stampa  risultati
# ...
print('Configurazione migliore:', best_cfg)
print('MAE valida:', best_mae)
print('MAE test:', test_mae)